In [ ]:
from bs4 import BeautifulSoup
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import json
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
options = Options()
# options.add_argument("--headless")
browser = webdriver.Chrome(options=options)
wait = WebDriverWait(browser, 10)
browser.get("https://d.weibo.com/")
browser.maximize_window()

In [ ]:
# 导入cookie
browser.delete_all_cookies()
cookie_filename = "./data/weibo_cookies.json"
cookies_file = open(cookie_filename, 'r', encoding='utf-8')
cookies = json.load(cookies_file)

for cookie in cookies:
    browser.add_cookie({
        'domain': cookie['domain'],
        'name': cookie['name'],
        'value': cookie['value'],
        'path': '/',
        'expires': None
    })

browser.get("https://d.weibo.com/")

In [ ]:
def user_input(box, word):
    for char in word:
        box.send_keys(char)
        time.sleep(0.1 +random.uniform(-0.06,0.06))
    box.send_keys(Keys.ENTER)
input_box = browser.find_element_by_class_name("W_input")
user_input(input_box, "武汉")

In [ ]:
date_list = []
for i in range(31):
    for j in range(24):
        date_list.append({'year':'2019', 'month':'11', 'day':i, 'hour':str(j)})
for i in range(12):
    monthday = 31
    if i == 2-1:
        monthday = 29
    elif i == 4-1 or i == 6-1 or i == 9-1 or i == 11-1:
        monthday = 30
    for j in range(monthday):
        for k in range(24):
            # date_list.append(f"2020-{str(i).rjust(2,'0')}-{str(j).rjust(2, '0')}-{str(k)}")
            date_list.append({'year':'2020', 'month':str(i), 'day':j, 'hour':str(k)})
import json
date_file = open("./data/date.json", 'w', encoding='utf-8')
json.dump(date_list, date_file)
date_file.close()

In [ ]:
item = date_list[100]
y = item['year']
m = item['month']
d = item['day']
hour = item['hour']

# 过程化测试代码
advs = browser.find_element_by_css_selector("a[node-type='advsearch']")
advs.click()
stime = browser.find_element_by_css_selector("input[node-type='stime']")
stime.click()
from selenium.webdriver.support.select import Select
s_month = Select(browser.find_element_by_class_name('month'))
s_month.select_by_value(m)
s_year = Select(browser.find_element_by_class_name('year'))
s_year.select_by_value(y)
day = browser.find_elements_by_css_selector("a[action-type='date']")
day[d].click()
s_hour = Select(browser.find_element_by_name('startHour'))
s_hour.select_by_value(hour)


item = date_list[101]
y = item['year']
m = item['month']
d = item['day']
hour = item['hour']

etime = browser.find_element_by_css_selector("input[node-type='etime']")
etime.click()
e_month = Select(browser.find_element_by_class_name('month'))
e_month.select_by_value(m)
e_year = Select(browser.find_element_by_class_name('year'))
e_year.select_by_value(y)
day = browser.find_elements_by_css_selector("a[action-type='date']")
day[d].click()
e_hour = Select(browser.find_element_by_name('endHour'))
e_hour.select_by_value(hour)

search_btn = browser.find_element_by_class_name('s-btn-a')
search_btn.click()

In [ ]:
from selenium.webdriver.support.select import Select
def change_time(index):
    advs = browser.find_element_by_css_selector("a[node-type='advsearch']")
    advs.click()

    item = date_list[index]
    y = item['year']
    m = item['month']
    d = item['day']
    hour = item['hour']

    stime = browser.find_element_by_css_selector("input[node-type='stime']")
    stime.click()
    s_month = Select(browser.find_element_by_class_name('month'))
    s_month.select_by_value(m)
    s_year = Select(browser.find_element_by_class_name('year'))
    s_year.select_by_value(y)
    day = browser.find_elements_by_css_selector("a[action-type='date']")
    day[d].click()
    s_hour = Select(browser.find_element_by_name('startHour'))
    s_hour.select_by_value(hour)
    time.sleep(0.1 +random.uniform(-0.06,0.06))

    item = date_list[index+1]
    y = item['year']
    m = item['month']
    d = item['day']
    hour = item['hour']

    etime = browser.find_element_by_css_selector("input[node-type='etime']")
    etime.click()
    e_month = Select(browser.find_element_by_class_name('month'))
    e_month.select_by_value(m)
    e_year = Select(browser.find_element_by_class_name('year'))
    e_year.select_by_value(y)
    day = browser.find_elements_by_css_selector("a[action-type='date']")
    day[d].click()
    e_hour = Select(browser.find_element_by_name('endHour'))
    e_hour.select_by_value(hour)

    search_btn = browser.find_element_by_class_name('s-btn-a')
    search_btn.click()

In [ ]:
s = len(date_list)-1 # 先排除掉最后一小时
for i in range(3, s):
    change_time(i)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    y = date_list[i]['year']
    m = date_list[i]['month']
    d = date_list[i]['day']
    h = date_list[i]['hour']
    with open(f'./out/{y}-{m}-{str(d+1)}-{h}.txt', 'w', encoding='utf-8') as f:
        for item in soup.select('p.txt'):
            print(item.get_text(), file=f)
    print(i, 'finished')

In [ ]:

s = len(date_list)-1 # 先排除掉最后一天
for day in range(s):
    for hour in range(23):
        start = date_list[day] + '-' + str(hour)
        end = date_list[day] + '-' + str(hour + 1)
        with open(f"./out/{date_list[day]}-{hour}.txt", 'w', encoding='utf-8') as f:
            for pagenum in range(1,51):
                browser.get(f"https://s.weibo.com/weibo?q=武汉&typeall=1&suball=1&timescope=custom:{start}:{end}&Refer=g&page={pagenum}")
                soup = BeautifulSoup(browser.page_source, 'html.parser')
                for item in soup.select('p.txt'):
                    print(item.get_text(), file=f)
                # time.sleep(0.1 +random.uniform(-0.06,0.06))
    # 接下来对23点到0点做特别爬取
    start = date_list[day] + '-24'
    end = date_list[day+1] + '-0'
    with open(f"./out/{date_list[day]}-{hour}.txt", 'w', encoding='utf-8') as f:
        for pagenum in range(1,51):
            browser.get(f"https://s.weibo.com/weibo?q=武汉&typeall=1&suball=1&timescope=custom:{start}:{end}&Refer=g&page={pagenum}")
            soup = BeautifulSoup(browser.page_source, 'html.parser')
            for item in soup.select('p.txt'):
                print(item.get_text(), file=f)
            # time.sleep(0.1 +random.uniform(-0.06,0.06))

In [ ]:
# 用于生成2019年的所有日期字符串
date_list = []
for i in range(1,32):
    for j in range(24):
        date_list.append(f"2019-12-{str(i).rjust(2, '0')}-{str(j)}")
for i in range(1, 13):
    monthday = 31
    if i == 2:
        monthday = 29
    elif i == 4 or i == 6 or i == 9 or i == 11:
        monthday = 30
    for j in range(1, monthday+1):
        for k in range(24):
            date_list.append(f"2020-{str(i).rjust(2,'0')}-{str(j).rjust(2, '0')}-{str(k)}")
date_file = open("./data/date.txt", 'w', encoding='utf-8')
l = len(date_list)
for i in range(1, l):
    print(date_list[i-1]+':'+date_list[i], file=date_file)
date_file.close()